# **Image Processing: Principal Component Analysis (PCA)**

In this Jupyter Notebook, we will implement Principal Compomnent Analysis (PCA) using NumPy, and conduct some experiments that are similar to the "EigenFace" study.

We use the sign language image data. What we are basically doing is to select the image data of one class (e.g., the sign for "zero"), and extract its principal components (PCs). Then we can visualize the data by reducing the dimension to the first 2 most dominant PCs. Also, given a new image, we can use its projections onto the PCs to determine whether it is a sign for "zero" or not.

***

## **1. Load Data**

Load data and preprocess.

The dimension of data $X$ is $n\times m$, in which $n=4096=64\times64$ is the total number of pixels in an image, and $m=163$ is the number of examples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
X_all = np.load(open('../data/X_train.npy', 'rb'))
Y_all = np.load(open('../data/Y_train.npy', 'rb'))
# print(X_all.shape)
# print(Y_all.shape)
# print(Y_all[162,]) # y=0
# print(Y_all[163,]) # y=1
# print(Y_all[327,]) # y=2

X_reshape = np.transpose(X_all, (1,2,3,0)).reshape(-1, X_all.shape[0])
X = X_reshape[:, :163]
print('X shape:', X.shape)

# Print some sample images
fig = plt.figure(figsize=(8, 4))
for i, idx in enumerate([0, 50, 100]):
    fig.add_subplot(1, 3, i+1)
    img = X_all[idx,:,:,0]
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.title('Sample image {}'.format(i+1))
plt.show()

## **2. Centerize Data**

We need to centerize the data by subtracting $X$ from the mean values of each row. I.e., for each $i,j$, $X[i,j]=mean(X[i,:])$.

This step is necessary before carrying out PCA. Notice that in the slides we do a further normalization by dividing by the standard deviation after subtracting the means. In this notebook, we do not use the dividing step for the convenience of later reconstruction steps. Fortunately, not fully normalizing the data will not harm PCA of image data.

- We can use `np.mean()` to compute the mean of each row, with argument `axis=1`
- The result of computing `np.mean` on $X$ (of shape (n, m)) could be (n,), and that means we may need to reshape it to (n,1) before subtracting it from $X$.

In [ ]:
X_tilde = X - np.mean(X, axis=1).reshape(-1, 1)

assert X.shape == X_tilde.shape
print('Shape of the centerized data:', X_tilde.shape)
print('First 5 elements of first row in X_tilde:', X_tilde[0,:5])
print('First 5 elements of last row in X_tilde:', X_tilde[-1,:5])

***

## **3. Construct Covariance Matrix and Compute Eigenvectors**

The principal components of data can be obtained by computing the eigenvectors $u_i$ of the covariance matrix $\frac{1}{m}\tilde{X}\tilde{X}^{T}$.

However, the shape of $\frac{1}{m}\tilde{X}\tilde{X}^{T}$ is $n\times n$, and it will be slow to carry out eigen decomposition when $n$ is big (In our case, $n=64\times 64=4096$). Therefore, we need to use an alternative way.

The **solution** is that we first compute the eigenvectors $u'_i$ of the matrix $\frac{1}{m}\tilde{X}^{T}\tilde{X}$ (of shape $m\times m$), which satisfy the following equation:

\begin{equation}
\big(\frac{1}{m}\tilde{X}^{T}\tilde{X}\big)u'_i = \lambda_i u'_i
\end{equation}

where $\lambda_i$ is the correponding non-zero eigenvalue. Note that $\lambda_i/\sum{\lambda_i}$ measures the proportion of variance that the $i$th principal component explains.
Because in our case $m=163$ is much smaller than $n=4096$, the computation for $u'_i$ is much faster. Then we can compute the eigenvectors of $\frac{1}{m}\tilde{X}\tilde{X}^{T}$ by: $u_i = \tilde{X}u'_i$. This is valid because of the following:

\begin{equation}
\big(\frac{1}{m}\tilde{X}^{T}\tilde{X}\big)u'_i = \lambda_i u'_i \Longrightarrow \tilde{X}\big(\frac{1}{m}\tilde{X}^{T}\tilde{X}\big)u'_i = \tilde{X}\lambda_i u'_i \Longrightarrow \big(\frac{1}{m}\tilde{X}\tilde{X}^{T}\big)(\tilde{X}u'_i) = \lambda_i (\tilde{X}u'_i)
\end{equation}

Of course we need to normalize $u_i$ by $u_i = \frac{u_i}{||u_i||}$, so that it satisfy the $||u||=1$ constraint of principal components.

- We can compute the eigenvectors of a $m\times m$ matrix by calling `numpy.linalg.eig()`. It returns two numpy arrays `w` and `v`, among which `w` contains $m$ eigenvalues, and `v` is a $m\times m$ array whose ith column `v[:,i]` is the corresponding eigenvector of `w[i]`  
- We can normalize a vector by calling `numpy.linalg.norm()`

In [ ]:
# Construct the covariance matrix for computing u'_i
covmat = (np.dot(X_tilde.T, X_tilde)) / X_tilde.shape[1]

# Compute u'_i, which is stored in the variable v
w, v = np.linalg.eig(covmat)

# Compute u_i from u'_i, and store it in the variable U
U = np.dot(X_tilde, v)

# Normalize u_i, i.e., each column of U
U = U / np.linalg.norm(U, axis=0)

# Evaluate eigenvalues
ratios = w / np.sum(w)
print('PC1 explains {}% of the total variance'.format(ratios[0]))
print('PC2 explains {}% of the total variance'.format(ratios[1]))
print('First 100 PCs explains {}% of the total variance'.format(sum(ratios[:100])))
print()

# Evaluate U
print('Shape of U:', U.shape)
print('First 5 elements of first column of U:', U[:5,0])
print('First 5 elements of last column of U:', U[:5,-1])

# Plot eigenvectors as if they are image data, i.e., eigenhands
fig = plt.figure(figsize=(8, 4))
for i in range(3):
    img = U[:, i].reshape((64, 64))
    fig.add_subplot(1, 3, i+1)
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.title('PC' + str(i+1))
plt.show()

***

## **4. Reduce Dimension and Visualize**

After we have obtained all the principal components, we can conduct dimensionality reduction by project the original data to a selected set of PCs. For visualization purpose, we select the top two PCs, and reduce the data to a 2-D space.

Let $u_1$ and $u_2$ the 1st and 2nd columns of matrix `U`, the projection of an image data $\tilde{x}$ (already centerized) can be done by computing the inner products $y_1 = u_1^T\tilde{x}$ and $y_2 = u_2^T\tilde{x}$. Then we can visualize the data using the scatter plot of $y_1$ and $y_2$.

- We select the first two PCs by `U[:,:2]`, and then use `numpy.dot()` to compute the inner product with $\tilde{X}$.
- `Y` is of shape $2\times m$.

In [ ]:
Y = np.dot(U[:, :2].T, X_tilde)

print('Shape of Y:', Y.shape)
print('First 5 elements of first row of Y:', Y[0,:5])
print('First 5 elements of second row of Y:', Y[1,:5])

fig = plt.figure()
plt.scatter(Y[0,:], Y[1,:])
plt.show()

***

## **5. Project and Reconstruct**

We can not only project data onto principal components to reduce dimensions, but also use the projections to reconstruct data. 

Let $\tilde{x}$ be the original data, and its projections on the first $k$ PCs are: $p_1 = u_1^T\tilde{x},p_2 = u_2^T\tilde{x},\dots,p_k = u_k^T\tilde{x}$. Then we can have the reconstruction of data $\tilde{x}'$ by:

\begin{equation}
\tilde{x}' = p_1\cdot u_1 + p_2\cdot u_2 + \dots + p_k\cdot u_k
\end{equation}

If the full set of PCs are used, then the reconstruction will be perfect, i.e., exactly the same as the original image without losing any informaiton. If a subset (e.g., top $k$ PCs) is used, then the reconstruction will cause some information loss. 

This information loss can be measured by the Euclidean distance between the original data $\tilde{x}$ and the reconstruted data $\tilde{x}'$. Larger distance indicates higher information loss.

- We will use `numpy.dot()` to compute projections and reconstructions.
- We will use `numpy.linalg.norm()` to compute distance.

In [ ]:
# Project and reconstruct
def project_reconstruct(X, U, num_dims):
    """
    Args:
    X -- centerized image data, of shape (n, m)
    U -- numpy array containing principal components, of shape (n, m)
    num_dims -- the number of PCs upon which to project X.
    
    Return:
    X_proj -- projections on the top num_dims PCs, of shape (num_dims, m)
    X_recon -- reconstructed data, of shape (n, m)
    dist -- Euclidean distance between X and X_recon, of shape (m,)
    """
    assert num_dims <= U.shape[1]
    assert X.shape[0] == U.shape[0]
    
    X_proj = np.dot(U[:, :num_dims].T, X)
    X_recon = np.dot(U[:, :num_dims], X_proj)
    dist = np.linalg.norm(X - X_recon, axis=0)
    
    return X_proj, X_recon, dist

In [ ]:
# Test code
fig = plt.figure(figsize=(12, 6))
fig.add_subplot(1, 4, 1)
img_orig = X_tilde[:,0].reshape((64, 64))
plt.imshow(img_orig, cmap='gray')
plt.axis('off')
plt.title('Original')

for i, num_dims in enumerate([163, 100, 50]):
    X_proj, X_recon, dist = project_reconstruct(X_tilde, U, num_dims)
    print('Mean distance of reconstruction using {} PCs: {}'.format(num_dims, np.mean(dist)))
    
    fig.add_subplot(1, 4, i+2)
    img_recon = X_recon[:,0].reshape((64, 64))
    plt.imshow(img_recon, cmap='gray')
    plt.axis('off')
    plt.title('Recon using {} PCs'.format(num_dims))
    
plt.show()

***

## **6. New Image Detection**

Given a new image of the sign for a different number, if we project it onto the PCs that we extracted above, then the reconstruction distance will be very large, and the reconstructed image will not likely be recognizable. This is because the principal components summarize current image class well, but not apply to images of different classes. 

This is also how PCA can be used in applications such as face detection etc.

In [ ]:
# Project and reconstruct one instance of new data
x_new = X_reshape[:, 163]
x_new_tilde = x_new - np.mean(X, axis=1)

_, x_new_recon, dist = project_reconstruct(x_new_tilde, U, 163)
print('Reconstruction distance for out-of-class data:', dist)

# Plot
fig = plt.figure(figsize=(6, 3))

fig.add_subplot(1, 2, 1)
img_orig = x_new_tilde.reshape((64, 64))
plt.imshow(img_orig, cmap='gray')
plt.axis('off')
plt.title('Original')

fig.add_subplot(1, 2, 2)
img_recon = x_new_recon.reshape((64, 64))
plt.imshow(img_recon, cmap='gray')
plt.axis('off')
plt.title('Recon using 163 PCs')

plt.show()